# BustaBit

Freeze time and analyze the board. We see the prevailing multiplier, we see the remaining players that have yet to lock. We see the total bet amount in the game as well as the highest bet (which tells us the bonus per bit).

With this data, we can analyze the expected value of each outcome.

## Bonus
The bonus per bit is (10%)*(total bets) / (largest bet). The bonus is assigned in order to the last players to lock (getting 1 unit of bonus per bit that they bet) until depleted.

## Expected Value
### Execute Now
If we locked right now, we would guarantee ourselves the multiplier and would have some probability of obtaining the bonus (really a distribution since we could get a partial bonus). This probability is based on the total amount bet by the remaining players relative to the amount of bonus to be deciminated. For example, if the sum of all remaining players bets is less than or equal to the amount of bonus, locking would give the bonus with probability one. If people default by delaying execution now, then earlier executers further down the queue will then take some of the bonus that would have been given to the defaulter had they executed in this moment.

### Delay Execution
If we choose to delay execution, then there is a default probability on our payout. However, we also stand to get a larger multiplier and potentially some bonus. In this case, we need to look at the probability of defaulting in the next delta t time step and the expected value of executing in the next time step versus delaying execution in the next time step. This is inductive and depends on the behavior of the other players. For this reason, we need to understand the likelihood of our opponents behaving in certain ways. Then, short Monte Carlo simulations can be run to estiamte the EV in real time.

### Execution Strategy
The optimal strategy will be to maximize expected value. That means at each step we will execute if that has higher EV than delaying execution. Otherwise, we will delay until executing has higher EV.

In [1]:
import numpy as np
import pandas as pd
import datetime
from scipy import optimize

In [2]:
def pdf(t):
    if t < 1:
        return 0
    return .99 / t**2

def cdf(t):
    if t < 1:
        return 0
    return 1. - .99/t

def cond_pdf(t, tstar=.99):
    '''Prob(T=t | T>=tstar)'''
    if t < tstar:
        return 0
    return float(tstar) / t**2

def cond_cdf(t, tstar=.99):
    '''Prob(T<=t | T>=tstar)'''
    if t < tstar:
        return 0
    return 1. - float(tstar) / t

In [ ]:
cdf(1.09)

In [3]:
def random_default_times(n, m=None):
    if m is None:
        u = np.random.rand(n)
    else:
        u = np.random.rand(n, m)
    t = .99 / (1. - u)
    return np.where(u <= .01, 1, t)

def cond_random_default_times(tstar, n, m=None):
    '''conditioned on t >= tstar'''
    if m is None:
        u = np.random.rand(n)
    else:
        u = np.random.rand(n, m)
    t = tstar / (1. - u)
    return t

In [ ]:
df = pd.read_csv('/Users/andingo/Desktop/bustabit2.csv')
df

In [ ]:
m = df.BET.max()
B = df.BET.sum()
u = np.array(df['@'])
s = np.array(df.BET)
N = 10000

In [ ]:
paths = cond_random_default_times(t, N)
t0 = 2.04
b0 = 1000
x0 = np.array([t0, b0])

def p(t, b, m, u, s, paths):
    J = u >= t
    tau = paths[np.newaxis, :]
    I = m - s[J].dot(u[J][:, np.newaxis] <= tau) >= b
    return I.mean()

def obj(x, *args):
    B, m, u, s, paths = args
    t, b = x
    pp = p(t, b, m, u, s, paths)
#     print '%10f\t%10f\t%10f' % (t, b, pp)
    
    return -.99*B*b / (10*m*t) * pp +\
        999999999*((t < 1.01) | (b < 1) | (b > 1e6) | (t > 300))

%time optimize.fmin(obj, x0, args=(B, m, u, s, paths))

In [4]:
import selenium.webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

In [5]:
firefox_profile = selenium.webdriver.FirefoxProfile('/Users/andingo/Library/Application Support/Firefox/Profiles/qgc8x8w3.Selenium')
wd = selenium.webdriver.Firefox(firefox_profile=firefox_profile)

In [6]:
wd.get('https://www.bustabit.com')

## Manually Log In and press Play Now

In [223]:
bet_button = wd.find_element_by_class_name('bet-button')

In [224]:
multiplier = wd.find_element_by_class_name('in-progress')

In [264]:
elem = wd.find_element_by_tag_name('tbody')
rows = elem.text.split('\n')
top_better = rows[0].split(' ')[0]
%time bets = [int(row.split(' ')[2].replace(',','')) for row in rows]
m = max(bets)
B = sum(bets)
bpb = .1*B/m
print top_better
print m, B, bpb

## TODO
# L and q will be a function of top_better
L = 2.2
# L = 1.3
q = .8
# q = .5

try:
    playing = wd.find_element_by_class_name('bet-bar-playing')
except NoSuchElementException:
    playing = None

CPU times: user 351 µs, sys: 4 µs, total: 355 µs
Wall time: 374 µs
TRoze
32851 100908 0.307168731545
CPU times: user 351 µs, sys: 4 µs, total: 355 µs
Wall time: 374 µs
TRoze
32851 100908 0.307168731545


In [265]:
while 'cashout' in bet_button.get_attribute('class'):
    try:
        if playing:
            style = playing.get_attribute('style')
    except StaleElementReferenceException:
        style = ''

    try:
        mult = float(multiplier.text[:-1])
    except StaleElementReferenceException:
        mult = 1.0
        
    try:
        in_play = float(style.split(' ')[1][:-2])/100 + .005
    except ValueError:
        in_play = 1.0
    except IndexError:
        in_play = 1.0
    if m >= B*in_play:
        bet_button.click()
        print 'A', m, B, in_play, B*in_play, mult
        break
    if m >= B*in_play - m:
        p = cond_cdf(L, mult)
        E_cash = mult-1.+bpb*p
        E_wait = (L-1.+bpb)*q*(1.-p)
#         print E_cash, E_wait
        if E_cash >= E_wait:
            bet_button.click()
            print 'B', m, B, in_play, B*in_play, mult, E_cash, E_wait
            break

In [268]:
# wd.quit()